In [2]:
## basic imports ##
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import math
import tensorflow as tf

# numpy imports
from numpy import load
from numpy import loadtxt
from numpy import nan
from numpy import isnan
from numpy import count_nonzero
from numpy import unique
from numpy import array

from sklearn.linear_model import SGDRegressor

import pandas as pd
# from kats.models.prophet import ProphetModel, ProphetParams
# from kats.consts import TimeSeriesData
# from kats.models.var import VARModel, VARParams
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import math
import numpy as np
import matplotlib.pyplot as plt

### Loading & Prepocessing data

In [3]:
def load_data(file_name, relative_path):
    

    # get save path 
    save_path = os.getcwd() +  relative_path # https://stackoverflow.com/questions/39125532/file-does-not-exist-in-jupyter-notebook
    full_path = os.path.join(save_path, file_name)
    assert  os.path.exists(full_path), f'Path {relative_path}/{file_name} does not exist'

    # read via panda reader
    df = pd.read_csv(full_path)
    return df

def preprocess_data(df):
        
    df = df.sample(frac=1)
    # separate into input and targets 
    targets = df.pop("out")
    # print("df", df)
    feature_names = list(df.columns)

    # # datapipeline at home
    # depth = 2
    # b = tf.one_hot(targets, depth)
    # b = b.numpy()
    # #print(b)
    # type(b)
    # targets = b#.tolist()

    df = df.to_numpy()
    # targets = targets.to_numpy()

    # Split the dataset into a train, test and validation split
    # ratio is 80:10:10
    train_ds, test_ds, validation_ds = np.split(df, [int(.8*len(df)), int(.9*len(df))])
    train_tar, test_tar, validation_tar = np.split(targets, [int(.8*len(targets)), int(.9*len(targets))])

    return (train_ds, train_tar), (test_ds,test_tar), (validation_ds,validation_tar)


### Model functions

In [4]:
def error_function( prediction, targets):
    error_term = 0.5*(prediction - targets)**2
    return error_term

def run_SGD(train_ds, test_ds):
    train_input, train_tar = train_ds
    test_input, test_tar = test_ds
    sgd_model = SGDRegressor(max_iter=100, tol=1e-3)
    sgd_model.fit(train_input, train_tar)
    # train vs test acc
    test_pred = sgd_model.predict(test_input) 		 
    train_pred = sgd_model.predict(train_input) 		

    test_error = error_function(test_pred, test_tar)
    train_error = error_function(train_pred, train_tar)
    return test_pred, test_error, train_error

def plot_train(prediction, target, name="SGD"):
    # #plot prediction vs real data
    x_line = np.arange(0, target.size)
    plt.plot(x_line, target, prediction)
    plt.legend(('real', 'predicted'))
    plt.title(f'target: {target.name}, model: {name}')
    plt.show()

### Main

In [5]:
## main
file_name = 'dummy_df.csv'
relative_path =   '/../data'

df = load_data(file_name, relative_path)
train_ds, test_ds, val_ds = preprocess_data(df)

test_pred, test_error, train_error = run_SGD(train_ds, test_ds)

plot_train(test_pred, test_ds[1])
print(test_error)
print(train_error)

ValueError: y should be a 1d array, got an array of shape (400, 2) instead.